## 専門知識労働者

###専門知識労働者である質問者が
###保険技術会社であるInsurellmの従業員が使用する
###エージェントは正確である必要があり、ソリューションは低コストでなければなりません。

このプロジェクトでは、RAG（検索拡張生成）を使用して、質問/回答アシスタントが高い精度を確保します。

In [ ]:
# 輸入

import os
import glob
from dotenv import load_dotenv
import gradio as gr

In [ ]:
# Langchainの輸入

from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

In [ ]:
# 価格は当社の要因であるため、低コストモデルを使用します

MODEL = "gpt-4o-mini"
db_name = "vector_db"

In [ ]:
# .envというファイルに環境変数をロードします

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')

In [ ]:
# Langchainのローダーを使用してドキュメントを読んでください
# 知識ベースのすべてのサブフォルダーにすべてを取ります

folders = glob.glob("knowledge-base/*")

# 一部のユーザーに必要なこの修正については、コースの学生であるCGとJon Rに感謝します
text_loader_kwargs = {'encoding': 'utf-8'}
# それがうまくいかない場合、一部のWindowsユーザーは代わりに次の行を除外する必要があるかもしれません
# text_loader_kwargs = {'autodetect_encoding'：true}

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        documents.append(doc)

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

In [ ]:
len(chunks)

In [ ]:
doc_types = set(chunk.metadata['doc_type'] for chunk in chunks)
print(f"Document types found: {', '.join(doc_types)}")

## 埋め込みのサイドノート、および「自動エンコードLLMS」

テキストの各塊を、埋め込みとして知られるテキストの意味を表すベクトルにマッピングします。

Openaiはこれを行うためのモデルを提供します。これをLangchainコードでAPIを呼び出すことで使用します。

このモデルは、完全な入力が与えられた出力を生成する「自動エンコードLLM」の例です。
「自動回帰LLM」として知られている今日説明した他のすべてのLLMとは異なり、過去のコンテキストのみに基づいて将来のトークンを生成します。

自動エンコードLLMの別の例は、GoogleのBertです。埋め込みに加えて、自動エンコードLLMが分類に使用されることがよくあります。

###サイドノート

第8週には、RAGおよびベクトルの埋め込みに戻り、オープンソースベクトルエンコーダを使用して、データがコンピューターを離れることはありません。これは、エンタープライズシステムとデータが内部を維持する必要がある場合に重要な考慮事項です。

In [ ]:
# データの塊を、各チャンクと埋め込むベクトルを関連付けるベクトルストアに入れます
# ChromaはSQLLITEに基づいた人気のオープンソースベクトルデータベースです

embeddings = OpenAIEmbeddings()

# あなたがむしろHuggingface文と変換者からのフリーベクトル埋め込みを使用したい場合
# 次に、埋め込み= openAiembedings（）を交換します
# と：
# langchain.embeddingsからImport Huggingfacembeddingsから
# Embeddings = HuggingfaceMbedings（model_name = "cente-transformers/all-minilm-l6-v2"）

# 既に存在する場合は削除します

if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

# VectorStoreを作成します

vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

In [ ]:
# 1つのベクトルを取得して、それが持っている寸法の数を見つけます

collection = vectorstore._collection
sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"The vectors have {dimensions:,} dimensions")

## ベクトルストアの視覚化

ドキュメントとその埋め込みベクトルを見て、何が起こっているのかを確認しましょう。

In [ ]:
# プレウォーク

result = collection.get(include=['embeddings', 'documents', 'metadatas'])
vectors = np.array(result['embeddings'])
documents = result['documents']
doc_types = [metadata['doc_type'] for metadata in result['metadatas']]
colors = [['blue', 'green', 'red', 'orange'][['products', 'employees', 'contracts', 'company'].index(t)] for t in doc_types]

In [ ]:
# 私たち人間は、2Dで物事を視覚化する方が簡単だと感じています！
# T-SNEを使用して、ベクターの寸法を2Dに減らします
# （t分配された確率的隣接埋め込み）

tsne = TSNE(n_components=2, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# 2D散布図を作成します
fig = go.Figure(data=[go.Scatter(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='2D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x',yaxis_title='y'),
    width=800,
    height=600,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

In [ ]:
# 3Dを試してみましょう！

tsne = TSNE(n_components=3, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# 3D散布図を作成します
fig = go.Figure(data=[go.Scatter3d(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    z=reduced_vectors[:, 2],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='3D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='z'),
    width=900,
    height=700,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

# Langchainを使用してすべてをまとめる時間です

<テーブルスタイル= "マージン：0;テキストアライグ：左;">
    <tr>
        <td style = "width：150px; height：150px; vertical-align：middle;">
            <img src = "../ fality.jpg" width = "150" height = "150" style = "display：block;" />
        </td>
        <td>
            <h2 style = "color：＃900;">私を読んでください！非推奨警告を無視する</h2>
            <span style = "color：＃900;">次のセルを実行すると、langchaindeprecationwarningが得られます 
            Langchainメモリを使用する簡単な方法について。彼らは私たちに、記憶のための新しいアプローチに移行するように頼みます。 
            私はこれについて非常に対立していると感じています。新しいアプローチには、ランググラフに移動し、エコシステムに深く入り込むことが含まれます。
            ランググラフにはかなりの量の学習とコーディングがありますが、率直に言って私たちの場合はそれほど利益はありません。<br/> <br/>
            コースにそれを組み込む方法/方法について考えますが、今のところは減価償却警告を無視してください
            そのままコードを使用します。 Langchainは、ConversationBufferMemoryをすぐに削除することは期待されていません。
            </span>
        </td>
    </tr>
</table>

In [ ]:
# Openaiとの新しいチャットを作成します
llm = ChatOpenAI(temperature=0.7, model_name=MODEL)

# チャットの会話メモリを設定します
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# レトリーバーは、ぼろきれに使用されるベクターストアを抽象化しています
retriever = vectorstore.as_retriever()

# それをまとめる：GPT 4o-Mini LLM、ベクトルストア、メモリで会話チェーンをセットアップ
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

In [ ]:
query = "Can you describe Insurellm in a few sentences"
result = conversation_chain.invoke({"question":query})
print(result["answer"])

In [ ]:
# チャットの新しい会話メモリを設定します
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# それをまとめる：GPT 4o-Mini LLM、ベクトルストア、メモリで会話チェーンをセットアップ
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

## チャットインターフェイスを使用して、これをグラデーションで紹介します - 

LLMとのチャットをプロトタイプする迅速かつ簡単な方法

In [ ]:
# 関数のラッピング - メモリがconversation_chainにあるため、履歴は使用されていないことに注意してください

def chat(message, history):
    result = conversation_chain.invoke({"question": message})
    return result["answer"]

In [ ]:
# そしてグラデーションで：

view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)